FIle.csv
File 
    method
    method
    
    
Readme

1) utils -> methode pour recuper tt les fichiers.py
2) utils -> toutes les noms de functions
3) main :
    loop 
        File
         method

In [1]:
pip install openai
pip install ast
pip install astor

SyntaxError: invalid syntax (3473095436.py, line 1)

In [ ]:
import os
import ast

def list_python_files(path):
    """
    List all Python files in the given directory path.
    """
    python_files = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".py"):
                python_files.append(os.path.join(root, file))
    return python_files


def get_functions_definitions(file_path):
    with open(file_path, 'r') as file:
        source_code = file.read()
        tree = ast.parse(source_code)
        function_definitions = []

        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef):
                function_code = ast.get_source_segment(source_code, node)
                function_definitions.append(function_code)

        return function_definitions




In [ ]:
list_python_files("workshop-docs-agent")


In [ ]:
mainframe = get_functions_definitions("workshop-docs-agent/resources/IIRA/gui/mainframe.py")
print(mainframe)

In [6]:
import ast

def inject_docstring(docstring, filename, function_name):
    with open(filename, 'r') as file:
        source = file.read()

    tree = ast.parse(source)

    def visit_FunctionDef(self, node):
        if node.name == self.function_name and not ast.get_docstring(node):
            # Split the docstring into lines and indent each line
            indented_lines = [(self.indent + line) if line else '' for line in self.docstring.splitlines()]
            # Join the lines back into a single string
            indented_docstring = '\n'.join(indented_lines)
            # Create a new AST node for the docstring with correct indentation
            docstring_node = ast.Expr(value=ast.Constant(value=indented_docstring, kind=None))
            # Insert the docstring at the beginning of the function body
            node.body.insert(0, docstring_node)
        return node

    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            visit_FunctionDef(node)

    modified_source = ast.unparse(tree)

    with open(filename, 'w') as file:
        file.write(modified_source)
    #how can i fix indentation    
    
docstring = ''' 
    This function does something.

    Parameters:
    arg1 (type): Description of arg1
    arg2 (type): Description of arg2

    Returns:
    type: Description of return value

'''
inject_docstring(docstring, 'testingfile.py', 'update_frame')  


In [19]:
import ast  

class DocstringInjector(ast.NodeTransformer):
    def __init__(self, function_name, docstring, indent='    '):  # Default indent is 4 spaces
        self.function_name = function_name
        self.docstring = docstring
        self.indent = indent

    def visit_ClassDef(self, node: ast.ClassDef):  
        if node.name == self.class_name and not ast.get_docstring(node):  
            # Split the docstring into lines and indent each line  
            indented_lines = [(self.indent + line) if line else '' for line in self.docstring.splitlines()]  
            # Join the lines back into a single string  
            indented_docstring = '\n'.join(indented_lines)  
            # Create a new AST node for the docstring with correct indentation  
            docstring_node = ast.Expr(value=ast.Constant(value=indented_docstring, kind=None))  
            # Insert the docstring at the beginning of the class body  
            node.body.insert(0, docstring_node)  
        return node      
    def visit_FunctionDef(self, node: ast.FunctionDef):
        if node.name == self.function_name and not ast.get_docstring(node):
            # Split the docstring into lines and indent each line
            indented_lines = [(self.indent + line) if line else '' for line in self.docstring.splitlines()]
            # Join the lines back into a single string
            indented_docstring = '\n'.join(indented_lines)
            # Create a new AST node for the docstring with correct indentation
            docstring_node = ast.Expr(value=ast.Constant(value=indented_docstring, kind=None))
            # Insert the docstring at the beginning of the function body
            node.body.insert(0, docstring_node)
        return node
    
def inject_docstring(docstring, filename, function_name):  
    with open(filename, 'r') as file:  
        source = file.read()  
  
    tree = ast.parse(source)  

    injector = DocstringInjector(function_name,docstring)
    modified_tree = injector.visit(tree)  
  
    modified_source = ast.unparse(modified_tree)  
  
    with open(filename, 'w') as file:  
        file.write(modified_source)  

docstring = ''' This function does something.

        Parameters:
        arg1 (type): Description of arg1
        arg2 (type): Description of arg2

        Returns:
        type: Description of return value

    '''  
inject_docstring(docstring, 'testingfile.py', 'update_frame')  
